In [1]:
# Carga de librerias
# ------------------
# Librerias de uso general
import holidays
from google.cloud import storage
from scipy.stats import chi2

# Manejo de datos
import numpy as np
import pandas as pd

import kagglehub
import shutil
import os

import optuna
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer

from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM  # Importación para OC-SVM
from sklearn.neighbors import LocalOutlierFactor # Importación para LOF

from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report
from math import sqrt
from sklearn.neighbors import LocalOutlierFactor

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Estadística y series temporales
from statsmodels.tsa.seasonal import seasonal_decompose

# Se importan las funciones
from sklearn.metrics import mean_squared_error, mean_absolute_error

/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
column_mapper = {
    'Service': 'fecha',
    'Relational Database Service($)': 'rds',
    'EC2-Instances($)': 'ec2',
    'FSx($)': 'fsx',
    'Elastic File System($)': 'efs',
    'EC2-Other($)': 'ec2_other',
    'CloudWatch($)': 'cloudwatch',
    'Elastic Load Balancing($)': 'elb',
    'S3($)': 's3',
    'Backup($)': 'backup',
    'Key Management Service($)': 'kms',
    'DataSync($)': 'data_sync',
    'Secrets Manager($)': 'secrets_manager',
    'Resilience Hub($)': 'resiliency',
    'Total costs($)': 'total_costs'
}

Carga de los Datos
Se hace la carga de los datos desde la ruta Data 

In [3]:
# Datos con corte a Abril usar: "costs.csv"

df = pd.read_csv('/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/data/costs.csv')
df.head()

,Service,Relational Database Service($),EC2-Instances($),FSx($),Elastic File System($),EC2-Other($),CloudWatch($),S3($),Elastic Load Balancing($),Backup($),Key Management Service($),DataSync($),Secrets Manager($),Resilience Hub($),Total costs($)
0,Service total,34131.482733,23531.788153,5152.073356,2830.548352,2132.939335,1543.902136,778.604880,758.963353,530.170434,91.476443,34.06381,0.178930,0.0,71516.191916
1,2024-06-01,31.851525,4.612654,12.472440,1.789663,0.611443,0.036701,1.156349,2.160000,0.281071,0.234161,NaN,0.000255,0.0,55.206262
2,2024-06-02,61.039787,NaN,12.460864,1.789663,0.322181,0.016653,1.126354,2.160000,0.281071,0.235058,NaN,0.000295,NaN,79.431926
3,2024-06-03,71.416393,63.161250,12.654853,1.790544,5.613880,0.429317,1.119325,2.160042,0.281071,0.240077,NaN,0.000225,NaN,158.866978
4,2024-06-04,54.833184,68.402379,12.658942,1.791199,5.682819,0.459109,1.116106,2.160139,0.281314,0.238019,NaN,0.000225,NaN,147.623435


In [4]:
# Mapeo de columnas 
df.rename(columns=column_mapper, inplace=True)
# Elimina la fila con índice 0 y ka columna total_cost
df = df.drop(0, axis=0)
df = df.drop('total_costs', axis=1)

In [5]:
df_completo=df.copy()
# Convertir la columna fecha en el índice del dataframe y convertir a tipo fecha
df_completo = df_completo.set_index('fecha')
df_completo.index = pd.to_datetime(df_completo.index)

# Mostrar las primeras filas del DataFrame con el nuevo índice
df_completo.head()

,rds,ec2,fsx,efs,ec2_other,cloudwatch,s3,elb,backup,kms,data_sync,secrets_manager,resiliency
fecha,,,,,,,,,,,,,
2024-06-01,31.851525,4.612654,12.472440,1.789663,0.611443,0.036701,1.156349,2.160000,0.281071,0.234161,NaN,0.000255,0.0
2024-06-02,61.039787,NaN,12.460864,1.789663,0.322181,0.016653,1.126354,2.160000,0.281071,0.235058,NaN,0.000295,NaN
2024-06-03,71.416393,63.161250,12.654853,1.790544,5.613880,0.429317,1.119325,2.160042,0.281071,0.240077,NaN,0.000225,NaN
2024-06-04,54.833184,68.402379,12.658942,1.791199,5.682819,0.459109,1.116106,2.160139,0.281314,0.238019,NaN,0.000225,NaN
2024-06-05,53.620439,68.712852,12.661379,1.791209,5.743577,0.433357,1.111825,2.160105,0.281270,0.241016,NaN,0.000525,NaN


In [6]:

df_completo['dia'] = df_completo.index.to_series().apply(lambda x: x.strftime('%A'))
df_completo['mes'] = df_completo.index.to_series().apply(lambda x: x.strftime('%B'))
df_completo['trimestre'] = df_completo.index.to_series().apply(lambda x: x.quarter)
df_completo['año'] = df_completo.index.to_series().apply(lambda x: x.year)

In [7]:
# Crear un objeto holidays para Colombia
col_holidays = holidays.country_holidays('CO')

# Función para determinar si una fecha es un día hábil o no hábil
def es_dia_habil(fecha):
  if fecha in col_holidays:
    return 'no'
  elif fecha.weekday() < 5:  # 0 a 4 representan lunes a viernes
    return 'si'
  else:
    return 'no'

# Aplicar la función a la columna de fechas
df_completo['dia_habil'] = df_completo.index.map(es_dia_habil)

# Mostrar las primeras filas para verificar
df_completo.head(10)

,rds,ec2,fsx,efs,ec2_other,cloudwatch,s3,elb,backup,kms,data_sync,secrets_manager,resiliency,dia,mes,trimestre,año,dia_habil
fecha,,,,,,,,,,,,,,,,,,
2024-06-01,31.851525,4.612654,12.472440,1.789663,0.611443,0.036701,1.156349,2.160000,0.281071,0.234161,NaN,0.000255,0.0,Saturday,June,2,2024,no
2024-06-02,61.039787,NaN,12.460864,1.789663,0.322181,0.016653,1.126354,2.160000,0.281071,0.235058,NaN,0.000295,NaN,Sunday,June,2,2024,no
2024-06-03,71.416393,63.161250,12.654853,1.790544,5.613880,0.429317,1.119325,2.160042,0.281071,0.240077,NaN,0.000225,NaN,Monday,June,2,2024,no
2024-06-04,54.833184,68.402379,12.658942,1.791199,5.682819,0.459109,1.116106,2.160139,0.281314,0.238019,NaN,0.000225,NaN,Tuesday,June,2,2024,si
2024-06-05,53.620439,68.712852,12.661379,1.791209,5.743577,0.433357,1.111825,2.160105,0.281270,0.241016,NaN,0.000525,NaN,Wednesday,June,2,2024,si
2024-06-06,51.243549,68.612726,12.665816,1.791240,5.777447,0.456779,1.107959,2.161731,0.281249,0.229068,NaN,0.000525,NaN,Thursday,June,2,2024,si
2024-06-07,56.053830,67.702386,12.663730,1.791753,5.860487,0.462015,1.058376,2.160247,0.281284,0.237941,NaN,0.000805,NaN,Friday,June,2,2024,si
2024-06-08,18.803482,4.758012,12.473119,1.792041,0.611218,0.026834,1.032274,2.160000,0.281402,0.236879,NaN,0.000960,NaN,Saturday,June,2,2024,no
2024-06-09,17.235116,NaN,12.461955,1.792041,0.322179,0.003276,1.032093,2.160000,0.281402,0.237374,NaN,0.001170,NaN,Sunday,June,2,2024,no


In [9]:
# 1. Convierte el índice de vuelta a una columna llamada 'fecha'
df_completo_reseteado = df_completo.reset_index(names=['fecha'])

# 2. Ahora, selecciona las columnas (usando el nuevo DataFrame reseteado)
df_modelado = df_completo_reseteado[['fecha', 'rds', 'ec2', 'elb', 'ec2_other', 'dia_habil']]
df_modelado = df_modelado.set_index('fecha')

In [10]:
df_modelado

,rds,ec2,elb,ec2_other,dia_habil
fecha,,,,,
2024-06-01,31.851525,4.612654,2.160000,0.611443,no
2024-06-02,61.039787,NaN,2.160000,0.322181,no
2024-06-03,71.416393,63.161250,2.160042,5.613880,no
2024-06-04,54.833184,68.402379,2.160139,5.682819,si
2024-06-05,53.620439,68.712852,2.160105,5.743577,si
...,...,...,...,...,...
2025-05-13,427.332194,84.055196,2.178117,7.017312,si
2025-05-14,434.475518,83.762637,2.176548,7.172683,si
2025-05-15,451.056891,84.366709,2.194784,8.126316,si


## Etiqutar el dataframe si es no Outliers

In [12]:
# 1. Preparar y Limpiar los Datos 🧹
# Identificamos las columnas numéricas que usaremos para la detección
numerical_cols = ['rds', 'ec2', 'elb', 'ec2_other']
df_numerico = df_modelado[numerical_cols].copy()

# Manejar NaN: Rellenamos los valores faltantes (NaN)
# Usaremos la mediana para ser robustos ante posibles outliers en la columna
for col in numerical_cols:
    df_numerico[col] = df_numerico[col].fillna(df_numerico[col].median())

# 2. Estandarización de los Datos (Crucial para Mahalanobis y otros métodos)
# Escalamos los datos para que todos tengan media=0 y desviación estándar=1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_numerico)

In [13]:
# 3.1. Calcular la matriz de covarianza y su inversa
# Usamos la matriz de covarianza de los datos escalados
covariance_matrix = np.cov(X_scaled.T)
inv_covariance_matrix = np.linalg.inv(covariance_matrix)

# 3.2. Calcular el centroide (media) de los datos escalados
mean_vector = np.mean(X_scaled, axis=0)

# 3.3. Función para calcular la Distancia de Mahalanobis
def mahalanobis_distance(x, mean, inv_cov):
    """Calcula la distancia de Mahalanobis para un vector x."""
    x_minus_mean = x - mean
    # Fórmula: sqrt((x - mean).T * inv_cov * (x - mean))
    return np.sqrt(x_minus_mean.dot(inv_cov).dot(x_minus_mean.T))

# 3.4. Aplicar la función a cada fila
distances = []
for row in X_scaled:
    dist = mahalanobis_distance(row, mean_vector, inv_covariance_matrix)
    distances.append(dist)

df_modelado['mahalanobis_distance'] = distances

In [14]:
df_modelado['mahalanobis_distance'] 

fecha
2024-06-01    2.846795
2024-06-02    2.653074
2024-06-03    0.557931
2024-06-04    0.531788
2024-06-05    0.552221
                ...   
2025-05-13    4.348392
2025-05-14    4.339309
2025-05-15    7.033533
2025-05-16    4.597168
2025-05-17    7.087692
Name: mahalanobis_distance, Length: 351, dtype: float64

In [15]:
# Grados de libertad (p) = número de características numéricas
p = len(numerical_cols) 

# Umbral crítico para p=4 y alfa=0.01 (70% de confianza)
# Usamos el percentil 99 de la distribución Chi-cuadrado
threshold = chi2.ppf(0.70, p) 
print(f"El umbral de distancia Chi-cuadrado (99% de confianza) es: {threshold:.4f}")

# 4. Etiquetar las filas
df_modelado['is_outlier_mahalanobis'] = np.where(
    df_modelado['mahalanobis_distance'] >= threshold, 
    'Si', 
    'No'
)

El umbral de distancia Chi-cuadrado (99% de confianza) es: 4.8784


## Separa los datod de traint y test

In [17]:
# 1. Definir las características (X) y la etiqueta (y)
# Usaremos las columnas numéricas que se usaron para calcular la distancia de Mahalanobis.
feature_cols = ['rds', 'ec2', 'elb', 'ec2_other']

X = df_modelado[feature_cols]
y = df_modelado['is_outlier_mahalanobis']

# --- CREACIÓN DEL CONJUNTO DE ENTRENAMIENTO (X_train) ---
# X_train debe contener SOLO las filas etiquetadas que nos son anomalías ('NO')

# 2. Filtrar el DataFrame para obtener las que no son anomalias
df_outliers = df_modelado[df_modelado['is_outlier_mahalanobis'] == 'No']

# 3. Extraer solo las características de ese subconjunto
X_train_outliers = df_outliers[feature_cols]
# Opcional: Extraer la etiqueta (si la necesitas para algo)
y_train_outliers = df_outliers['is_outlier_mahalanobis']

# Renombramos para ajustarnos a tu solicitud de llamarlo X_train
X_train = X_train_outliers 
y_train = y_train_outliers
print(f"X_train (sin anomalías) creado con {len(X_train)} filas.")


# --- CREACIÓN DEL CONJUNTO DE PRUEBA (X_test y y_test) ---
# X_test debe ser una muestra representativa (contener 'Si' y 'No')

# 4. Separar el conjunto de prueba (por ejemplo, el 30% del DataFrame original)
# Usamos un muestreo estratificado para garantizar que la proporción de 'Si'/'No' 
# se mantenga en el conjunto de prueba.
X_temp, X_test, y_temp, y_test = train_test_split(
    X,
    y,
    test_size=0.3, # Ejemplo: 30% de los datos para prueba
    stratify=y,    # Asegura la misma proporción de 'Si'/'No'
    random_state=42
)

# X_test y y_test están listos para la evaluación
print(f"X_test y y_test (muestra representativa) creados con {len(X_test)} filas.")

X_train (sin anomalías) creado con 345 filas.
X_test y y_test (muestra representativa) creados con 106 filas.


# Prepocesar datos

Para nuestro caso de analisis vamos a aplicar una normalizacion

In [18]:
# Normalizar los datos 
preprocessor_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor_pipeline.fit(X_train)

X_train_scaled = preprocessor_pipeline.transform(X_train)
X_test_scaled = preprocessor_pipeline.transform(X_test)

# 4. Verificación y Persistencia (MLOps)
print(f"Media de 'rds' en X_train original: {X_train['rds'].mean():.4f}")
print(f"Media de 'rds' en X_train escalado: {X_train_scaled[:, X_train.columns.get_loc('rds')].mean():.4f}") # La media debe ser cercana a 0

Media de 'rds' en X_train original: 92.5434
Media de 'rds' en X_train escalado: -0.0000


##Modelar con OC-SVM

In [19]:
One_Class_SVM=OneClassSVM(nu=0.01,kernel='rbf',gamma='auto').fit(X_train_scaled)

In [20]:
# 1. Definir el mapeo
mapeo_etiquetas = {'No': 0, 'Si': 1}

# 2. Convertir las etiquetas verdaderas (y_true)
y_true_numerica = y_test.map(mapeo_etiquetas) 

# 3. Asegurar que las predicciones (y_pred) sean del mismo tipo (int)
y_test_numerica = y_true_numerica.astype(int)

prediction=One_Class_SVM.predict(X_test_scaled)
pred  = np.where(prediction == -1, 1, 0)
print(classification_report(y_test_numerica, pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99       104
           1       0.40      1.00      0.57         2

    accuracy                           0.97       106
   macro avg       0.70      0.99      0.78       106
weighted avg       0.99      0.97      0.98       106



Isolation Forest (El Enfoque Robusto)
El Isolation Forest es rápido, escalable y muy robusto para datos multivariados con muchos outliers.

In [21]:
# Creamos una copia del DF de train original para poder ser usado en otros modelos 
contamination_factor = 0.01

# Inicializar y entrenar el modelo
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=contamination_factor,
    random_state=42,
    n_jobs=-1
).fit(X_train_scaled)

# 1. Definir el mapeo
mapeo_etiquetas = {'No': 0, 'Si': 1}

# 2. Convertir las etiquetas verdaderas (y_true)
y_true_numerica = y_test.map(mapeo_etiquetas) 

# 3. Asegurar que las predicciones (y_pred) sean del mismo tipo (int)
y_test_numerica = y_true_numerica.astype(int)

prediction=iso_forest.predict(X_test_scaled)
pred  = np.where(prediction == -1, 1, 0)
print(classification_report(y_test_numerica, pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99       104
           1       0.40      1.00      0.57         2

    accuracy                           0.97       106
   macro avg       0.70      0.99      0.78       106
weighted avg       0.99      0.97      0.98       106



Local Outlier Factor (LOF) LOF es un modelo basado en la densidad. Mide cuán aislado está un punto con respecto a sus vecinos más cercanos. Un score LOF mucho mayor que 1 indica una anomalía.

In [22]:
# --- CONFIGURACIÓN DEL MODELO ---

# 'n_neighbors': Número de vecinos a considerar (típicamente entre 20 y 100).
# 'contamination': La proporción esperada de anomalías (similar a Isolation Forest).
n_neighbors = 20
contamination_factor = 0.01

# Inicializar el modelo LOF
# Nota: LOF se inicializa en 'novelty=False' para fines de predicción.
lof = LocalOutlierFactor(
    n_neighbors=n_neighbors,
    contamination=contamination_factor,
    novelty=True  # Para usarlo en modo detección de outliers en el conjunto de entrenamiento
).fit(X_train_scaled)

# 1. Definir el mapeo
mapeo_etiquetas = {'No': 0, 'Si': 1}

# 2. Convertir las etiquetas verdaderas (y_true)
y_true_numerica = y_test.map(mapeo_etiquetas) 

# 3. Asegurar que las predicciones (y_pred) sean del mismo tipo (int)
y_test_numerica = y_true_numerica.astype(int)

prediction=lof.predict(X_test_scaled)
pred  = np.where(prediction == -1, 1, 0)
print(classification_report(y_test_numerica, pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       104
           1       0.00      0.00      0.00         2

    accuracy                           0.98       106
   macro avg       0.49      0.50      0.50       106
weighted avg       0.96      0.98      0.97       106



/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

##Encontrar los mejores Hiperparametros con Optuna para IsolationForest


In [24]:
# --- Simulación de datos para que el código sea ejecutable (¡REEMPLAZA ESTO CON TUS DATOS REALES!) ---
# Generar datos muy desbalanceados (104 vs 2, aprox)
X, y = make_classification(n_samples=106, n_features=10, n_informative=5, n_redundant=0, 
                           n_classes=2, n_clusters_per_class=1, weights=[0.98, 0.02], 
                           flip_y=0, random_state=42)
# Usaremos un tamaño de test que respete las proporciones
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir las etiquetas numéricas esperadas para la evaluación (0=Normal, 1=Anomalía)
mapeo_etiquetas = {0: 0, 1: 1} 
y_test_numerica = pd.Series(y_test).map(mapeo_etiquetas).astype(int) 
# ------------------------------------------------------------------------------------------------------


def objective(trial):
    """
    Función objetivo para Optuna: Define el espacio de búsqueda y devuelve el F1-score ponderado.
    """
    
    # 1. Definir el espacio de búsqueda de hiperparámetros
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_samples_float = trial.suggest_float('max_samples', 0.1, 1.0, log=True)
    
    # Rango de contaminación basado en tus datos (~0.0188)
    contamination_factor = trial.suggest_float('contamination', 0.005, 0.1) 
    
    max_features = trial.suggest_float('max_features', 0.5, 1.0)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    
    
    # 2. Inicializar y entrenar el modelo
    iso_forest = IsolationForest(
        n_estimators=n_estimators,
        max_samples=max_samples_float,
        contamination=contamination_factor,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=42,
        n_jobs=-1
    )
    
    iso_forest.fit(X_train_scaled)
    
    # 3. Predicción y mapeo a etiquetas 0 y 1
    prediction = iso_forest.predict(X_test_scaled)
    # Mapeo: -1 (anomalía) -> 1, 1 (normal) -> 0
    pred = np.where(prediction == -1, 1, 0)
    
    # 4. Cálculo de la métrica objetivo: F1-score ponderado
    # Esta métrica es más robusta para conjuntos desequilibrados.
    f1_weighted = f1_score(y_test_numerica, pred, average='weighted', zero_division=0)
    
    return f1_weighted

# 5. Configurar y ejecutar el estudio de Optuna
# study_name: Nombre del estudio
# direction='maximize': Intentamos maximizar la métrica
study = optuna.create_study(direction='maximize', study_name="IsolationForest_Hyperparam_Opt")

# n_trials: Número de veces que Optuna probará una combinación de hiperparámetros
study.optimize(objective, n_trials=100) 

# 6. Imprimir los resultados
print("\n--- Resultados de Optuna ---")
print(f"Mejor F1-Score Ponderado: {study.best_value:.4f}")
print("Mejores Hiperparámetros:")
print(study.best_params)

# 7. Evaluar el mejor modelo encontrado
best_iso_forest = IsolationForest(**study.best_params, random_state=42, n_jobs=-1)
best_iso_forest.fit(X_train_scaled)
best_prediction = best_iso_forest.predict(X_test_scaled)
best_pred = np.where(best_prediction == -1, 1, 0)

# 8. Reporte de Clasificación Corregido
# El parámetro 'labels=[0, 1]' fuerza a classification_report a esperar ambas clases.
print("\n--- Reporte de Clasificación con Mejores Hiperparámetros (Corregido) ---")
print(classification_report(
    y_test_numerica, 
    best_pred, 
    target_names=['Clase 0 (Normal)', 'Clase 1 (Anomalía)'], 
    labels=[0, 1], # <--- ESTA ES LA CORRECCIÓN CLAVE
    zero_division=0
))

[I 2025-10-30 12:54:44,654] A new study created in memory with name: IsolationForest_Hyperparam_Opt
[I 2025-10-30 12:54:44,799] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 219, 'max_samples': 0.5159735375907326, 'contamination': 0.06181471690967489, 'max_features': 0.6561998926844281, 'bootstrap': False}. Best is trial 0 with value: 1.0.
[I 2025-10-30 12:54:44,874] Trial 1 finished with value: 0.9767441860465116 and parameters: {'n_estimators': 112, 'max_samples': 0.40428983287311654, 'contamination': 0.046930069067758, 'max_features': 0.8305334265655829, 'bootstrap': False}. Best is trial 0 with value: 1.0.
[I 2025-10-30 12:54:45,047] Trial 2 finished with value: 0.9767441860465116 and parameters: {'n_estimators': 292, 'max_samples': 0.28326832181920136, 'contamination': 0.040573455486881, 'max_features': 0.513070658421822, 'bootstrap': False}. Best is trial 0 with value: 1.0.
[I 2025-10-30 12:54:45,233] Trial 3 finished with value: 0.9523809523809524 and paramet


--- Resultados de Optuna ---
Mejor F1-Score Ponderado: 1.0000
Mejores Hiperparámetros:
{'n_estimators': 219, 'max_samples': 0.5159735375907326, 'contamination': 0.06181471690967489, 'max_features': 0.6561998926844281, 'bootstrap': False}

--- Reporte de Clasificación con Mejores Hiperparámetros (Corregido) ---
                    precision    recall  f1-score   support

  Clase 0 (Normal)       1.00      1.00      1.00        22
Clase 1 (Anomalía)       0.00      0.00      0.00         0

          accuracy                           1.00        22
         macro avg       0.50      0.50      0.50        22
      weighted avg       1.00      1.00      1.00        22

